# Libraries

## To do
- Połączyć dane z kaggle
- Zrobić na tabeli per gość

## Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
from lxml import etree
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep
import re
from docx import Document
import nltk
from itertools import chain
import heapq
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import string

### Stopwords

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\szmar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\szmar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data import and transformation

## Import the Word documents

In [3]:
document_list = []
for i in range(9):
    document = Document('data/raw/Podcast_9{}.docx'.format(i))
    document_list.append(document)

### Define a function to extract headings

In [4]:
def iter_headings(paragraphs):
    for paragraph in paragraphs:
        if paragraph.style.name.startswith('Heading'):
            yield paragraph

### Proces the documents

In [19]:
results = pd.DataFrame()

for z in document_list:
    
    # Create an empty list for all headings
    heading_list = []
    
    # Extraction headings from this document
    for heading in iter_headings(z.paragraphs):
        heading_list.append(heading.text)

    # Remove the intro and outro
    heading_list = heading_list[1:-1]
    
    text_lines = []

    for i,para in enumerate(z.paragraphs):
        if para.text in heading_list:
            text_lines.append(i)

    #extracting text lines for games only (without intro and outro)       
    gamesText = []
    for k,j in enumerate(text_lines):
        for y, parar in enumerate(z.paragraphs):
            if y>=text_lines[k]:
                gamesText.append(parar.text)

    #creating list mapping where we get the in-text position of heading (number of line)            
    text_lines2 = [text_lines[n]-text_lines[0] for n in range(1,len(text_lines))]
    text_lines2.insert(0,0)
    
    #Extracting text with respect to headings: list of dialogs related to specific game
    extracted_text = []

    for d,w in zip(text_lines2,text_lines2[1:]):
        text = gamesText[d+1:w]
        extracted_text.append(text) 
    
    for game, text in zip(heading_list, extracted_text):
        for sentence in text:
            results = results.append({'game' : game, 'text': sentence}, ignore_index = True)

### Make changes to the text

In [21]:
# Remove the timestamp
results['game'] = results['game'].map(lambda x: str(x)[:-11])

# Split the text into person and speech
new = results["text"].str.split(":", n = 1, expand = True) 
  
# Extract the person speaking
results["person"] = new[0] 
  
# And what they are saying
results["text"] = new[1] 

# Drop if the formating was off
results.dropna(inplace = True)

# Just a preacusion
results[results['person'].map(len) < 12]

# Remove the part after their name
results["person"] = results["person"].str.split(" ", n = 1, expand = True)[0]

# Remove "This" and "And"
results = results[(results['person'] != 'This') & (results['person'] != 'And')]

# One of the games is wrong
results.replace('Twilight Imperium stream plug', 'Twilight Imperium', inplace = True)

# Create a list of games
games = results.game.unique()

In [22]:
results

,game,text,person
0,Blackout: Hong Kong,"Let’s talk briefly about Blackout: Hong Kong,...",Quinns
1,Blackout: Hong Kong,"And the reasons you were excited about it, yo...",Matt
2,Blackout: Hong Kong,This is really more of a public service annou...,Quinns
3,Blackout: Hong Kong,I think “exciting to try” is the key thing fo...,Matt
4,Blackout: Hong Kong,"If you haven’t played Great Western Trail, th...",Quinns
...,...,...,...
3470,Too Many Bones,Yup.,Quinns
3471,Too Many Bones,"And it’s like, cool! Good!",Matt
3472,Too Many Bones,"I mean, yeah, that’s the number one thing abo...",Quinns
3473,Too Many Bones,"Yeah. And also, why do you have to be boring?...",Matt


### Create per-game text table

In [55]:
grouped_results = dict()

for index, row in results.iterrows():
    game = row[0]
    text = row[1]
    if game not in grouped_results:
        grouped_results[game] = text
    else:
        grouped_results[game] = grouped_results[game] + ' ' + text
        
grouped_results = pd.DataFrame(list(grouped_results.items()), columns=['game', 'text'])

In [56]:
grouped_results

,game,text
0,Blackout: Hong Kong,"Let’s talk briefly about Blackout: Hong Kong,..."
1,Just One,Let’s talk about something a bit more positiv...
2,Sumer,"Let’s shift gears completely, Matthew. Let’s ..."
3,The Quacks of Quedlinburg,Let’s talk about a game that has been excitin...
4,Schummel Hummel,"Okay, let’s talk about a little card game, wh..."
...,...,...
61,Silk,So I’ve been playing a little game called Sil...
62,Pandemic: Rapid Response,I’ve played a couple of real-time co-op games...
63,Rail Pass,What I did have a great time with is somethin...
64,Crokinole,Let’s talk a little bit just about the review...


## Text transformation

### For now - each game as one text

### Tokenization

In [114]:
grouped_results['formated_text'] = grouped_results['text'].map(lambda x: x.split())

### Remove punctuation and stop words

In [118]:
# Stopwords
def remove_words(text):
    keep = []
    stop_words = list(stopwords.words('english')) + ['I', 'i', 'a', 'A', "it's"]
    punctuation = list(string.punctuation) + ["''", "_", "“", "’", "…", "[", "-", "”", "–"]
    
    for word in text:
        word = word.lower()
        if word not in stop_words:
            clean_word = ''
            for letter in word:
                if letter not in punctuation:
                    clean_word += letter.lower()
                    
            if clean_word != '': 
                keep.append(clean_word)      
    return keep

grouped_results['formated_text'] = grouped_results['formated_text'].map(remove_words)

### Calculate word frequency for all reviews

In [119]:
frequency = {}

for index, row in grouped_results.iterrows():
    text = row[2]
    
    for word in text:
        if word not in frequency.keys():
            frequency[word] = 1
        else:
            frequency[word] += 1

# Convert to a dataframe
frequency = pd.DataFrame(list(frequency.items()), columns=['word', 'frequency'])

# Sort by frequency
frequency.sort_values('frequency', ascending = False, inplace = True)

In [120]:
frequency

,word,frequency
88,like,1377
14,game,971
74,yeah,933
66,really,625
49,youre,513
...,...,...
3538,elf,1
3541,dudes,1
3542,intend,1
3545,cooler,1


## Webscrapping

In [7]:
headers = {'User-Agent': 'Text mining'}

# Construct the results table
boardgameList = ['Just One']
listhashed = [i.replace(' ', '%20') for i in boardgameList]
summaryTable = pd.DataFrame()

for j in listhashed:
    # Load the webpage
    
    r = requests.get('https://boardgamegeek.com/geeksearch.php?action=search&objecttype=boardgame&q={}'.format(j), headers=headers)
    
    # Convert to BS
    bs = BeautifulSoup(r.text)
    
    # Find the right table
    table = bs.findAll(lambda tag: tag.name=='table' and tag.has_attr('cellpadding'))
    #mydivs = bs.findAll("div", {"class": "table-responsive"})
    errors = bs.findAll(text="No Items Found")
   
    if not errors:
           
        tableHTML = table[0]
        #tableHTML = mydivs[0]
        # Convert the HTML of the table to pandas (very usefull function)
        pandasTable = pd.read_html(str(tableHTML), header=0)[0]

        # Extract all the links
        urls = [np.where(tag.has_attr('href'),tag.get('href'),"no link") for tag in tableHTML.find_all('a')]
        urls = pd.DataFrame(urls)
        urls = urls[0][6]

        # Merge the links into the DF
        pandasTable['link'] = urls

        # Drop unused columns
        pandasTable = pandasTable.drop(['Unnamed: 1', 'Shop'], axis = 1)

        # Create or extend the existing summary table
        if summaryTable.empty:
            summaryTable = pandasTable
        else:
            summaryTable = pd.concat([summaryTable, pandasTable],ignore_index=True)
    else:
        next

In [8]:
display(summaryTable)

,Board Game Rank,Title,Geek Rating,Avg Rating,Num Voters,link
0,219.0,Just One (2018),7.209,7.61,6196,/boardgame/254640/just-one
1,NaN,Port Royal: Just One More Contract... (2015),6.549,7.53,1461,/boardgame/254640/just-one
2,NaN,Just One Die Soccer (2009),NaN,1.00,1,/boardgame/254640/just-one
3,NaN,Just One: Deutscher Spielepreis Classic Goodie...,NaN,6.64,7,/boardgame/254640/just-one
4,NaN,Just One: Gathering of Friends Promo Cards (2019),NaN,7.00,3,/boardgame/254640/just-one
5,NaN,Cocks Abreast Hostility: Cock Pack One – Just ...,NaN,7.71,7,/boardgame/254640/just-one


In [9]:
# Loading the firefox webdriver
driver = webdriver.Chrome('/Users/korneliusz.krysiak/Dropbox/TextMining/chromedriver')

In [10]:
# Construct the results table
detailedTable = pd.DataFrame(columns=['Title','Average Rating','Number of ratings','Std.dev','Number of Comments','Fans','Page Views','Overall Ranking','All Time Plays','Played This Month', 'Own', 'Previously Owned','For Trade','Want in Trade','Wishlist','Has Parts','Want Parts'])
detailedTable2 = pd.DataFrame()

for j in range(1):
    # Load the webpage
    driver.get('https://boardgamegeek.com{}/stats'.format(summaryTable['link'][j]))
    
    # Get the source code and make it BS
    pageSource = driver.page_source
    bs = BeautifulSoup(pageSource, "lxml")
    
    # Find the relevant table
    data = bs.findAll('div',{'class':'outline-item-description'})
    
    # Loop over and strip spaces
    data2 = []
    for i in data:
        data2.append(i.get_text().strip())

    # Remove the ',' 
    for i in range(len(data2)):
        data2[i] = data2[i].replace(",","")

    # Wrangle the columns
    data2[3] = data2[3][0:4]
    data2[0] = float(data2[0])
    data2[1] = int(data2[1]) 
    data2[2] = float(data2[2])
    data2[3] = float(data2[3]) 

    # Convert to int
    for i in range(len(data2[4:])+1):
        i = i+3
        data2[i] = int(data2[i])

    # Special case - two categories. We want to drop both
    if len(data2) == 19:
        data2.pop(8)
        data2.pop(8)
    else:
        data2.pop(8)
    
    # Insert the data
    data2.insert(0,summaryTable['Title'][j]) 
    detailedTable.loc[len(detailedTable)] = data2

ValueError: invalid literal for int() with base 10: '53  Find For-Trade Matches'

In [11]:
display(detailedTable)

,Title,Average Rating,Number of ratings,Std.dev,Number of Comments,Fans,Page Views,Overall Ranking,All Time Plays,Played This Month,Own,Previously Owned,For Trade,Want in Trade,Wishlist,Has Parts,Want Parts


In [ ]:
# twoja kolej Michal